I want to group by HCPCS code for each payment type. Average the allowed payment (also do with mean to see if there's a difference at that level before proceeding). End up with 3 rows per HCPCS per payment type since it's still broken out by year. Then come up with the change over time by matching rows using HCPCS and payment type, and subtracting the year column value.
- potential issue: null values in 2015 column, how to write code so I get the 2017-2016 value? Would a for loop work here?
- conceptually this seems easier with the pivoted dataframe, but the aggregation involved in pivoting gets in the way
- may be helpful to create separate dataframes for each payment type - if I do this from the pivoted dataframe would that help with the aggregation issues?

In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline

In [2]:
df_payments_combined = pd.read_pickle('../data/payments_combined.pkl')

In [3]:
df_payments_combined.head()

,national_provider_identifier,last_name_organization_name_of_the_provider,entity_type_of_the_provider,city_of_the_provider,zip_code_of_the_provider,state_code_of_the_provider,provider_type,place_of_service,hcpcs_code,hcpcs_description,number_of_services,number_of_medicare_beneficiaries,number_of_distinct_medicare_beneficiary_per_day_services,average_medicare_allowed_amount,year,payment_type
0,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99217,Hospital observation care discharge,23.0,23.0,23.0,72.680000,2015,Doctor Only
1,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99219,Hospital observation care typically 50 minutes,18.0,18.0,18.0,135.850000,2015,Doctor Only
2,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99221,"Initial hospital inpatient care, typically 30 ...",59.0,58.0,59.0,101.365085,2015,Doctor Only
3,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99222,"Initial hospital inpatient care, typically 50 ...",132.0,130.0,132.0,139.010455,2015,Doctor Only
4,1003000126,ENKESHAFI,I,CUMBERLAND,215021854,MD,Internal Medicine,F,99223,"Initial hospital inpatient care, typically 70 ...",220.0,215.0,220.0,205.185955,2015,Doctor Only


In [4]:
df_pmt_pvt = pd.read_pickle('../data/df_pmt_pvt.pkl')

In [5]:
df_pmt_pvt.head()

year,national_provider_identifier,entity_type_of_the_provider,place_of_service,payment_type,provider_type,hcpcs_code,hcpcs_description,zip_code_of_the_provider,state_code_of_the_provider,2015,2016,2017
0,1003000126,I,F,Doctor Only,Internal Medicine,99217,Hospital observation care discharge,215021854,MD,72.680000,72.743158,73.398800
1,1003000126,I,F,Doctor Only,Internal Medicine,99218,Hospital observation care typically 30 minutes,215021854,MD,NaN,NaN,100.080000
2,1003000126,I,F,Doctor Only,Internal Medicine,99219,Hospital observation care typically 50 minutes,215021854,MD,135.850000,135.010000,136.380000
3,1003000126,I,F,Doctor Only,Internal Medicine,99220,Hospital observation care typically 70 minutes...,215021854,MD,NaN,189.239565,190.363729
4,1003000126,I,F,Doctor Only,Internal Medicine,99221,"Initial hospital inpatient care, typically 30 ...",215021854,MD,101.365085,100.750000,101.680000


In [6]:
df_pmt_pvt['diff_1615'] = df_pmt_pvt.2016 - df_pmt_pvt.2015

SyntaxError: invalid syntax (<ipython-input-6-902c62f9fe32>, line 1)

In [7]:
df_avg_pmt = df_payments_combined.groupby(['year',
                                           'payment_type',
                                           'hcpcs_code']).average_medicare_allowed_amount.mean().to_frame()

In [9]:
df_avg_pmt = df_avg_pmt.reset_index()
df_avg_pmt.head()

,year,payment_type,hcpcs_code,average_medicare_allowed_amount
0,2015,Doctor & Facility,00103,255.076212
1,2015,Doctor & Facility,00104,111.614526
2,2015,Doctor & Facility,00140,139.047955
3,2015,Doctor & Facility,00142,102.570003
4,2015,Doctor & Facility,00160,211.812813


In [10]:
df_avg_pmt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28879 entries, 0 to 28878
Data columns (total 4 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   year                             28879 non-null  int64  
 1   payment_type                     28879 non-null  object 
 2   hcpcs_code                       28879 non-null  object 
 3   average_medicare_allowed_amount  28879 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 902.6+ KB


In [77]:
df_avg_pmt.payment_type.value_counts()

Doctor & Facility    12689
Doctor Only          12119
Facility Only         4071
Name: payment_type, dtype: int64

In [11]:
df_avg_pmt.describe()

,year,average_medicare_allowed_amount
count,28879.00000,28879.000000
mean,2016.00980,379.808995
std,0.81578,1192.709320
min,2015.00000,0.010000
25%,2015.00000,33.868522
50%,2016.00000,124.781988
75%,2017.00000,388.973461
max,2017.00000,39279.086545


In [12]:
pivot_index = ['payment_type',
               'hcpcs_code']

pivot_cols = ['year']

In [13]:
df_avg_pmt_pvt = df_avg_pmt.pivot_table(index = pivot_index, 
                                              columns = pivot_cols, 
                                              values = 'average_medicare_allowed_amount').reset_index()

# df_avg_pmt_pvt = df_avg_pmt_pvt.reset_index()

In [14]:
df_avg_pmt_pvt.head()

year,payment_type,hcpcs_code,2015,2016,2017
0,Doctor & Facility,0008M,NaN,3419.419776,3443.360000
1,Doctor & Facility,00103,255.076212,128.268843,118.002858
2,Doctor & Facility,00104,111.614526,75.184242,74.647106
3,Doctor & Facility,0010M,NaN,333.559722,NaN
4,Doctor & Facility,00120,NaN,186.200899,142.635455


In [20]:
df_avg_pmt_pvt['change_2017_2015'] = df_avg_pmt_pvt[2017] - df_avg_pmt_pvt[2015]

In [21]:
df_avg_pmt_pvt.head()

year,payment_type,hcpcs_code,2015,2016,2017,change_2017_2015
0,Doctor & Facility,0008M,NaN,3419.419776,3443.360000,NaN
1,Doctor & Facility,00103,255.076212,128.268843,118.002858,-137.073353
2,Doctor & Facility,00104,111.614526,75.184242,74.647106,-36.967421
3,Doctor & Facility,0010M,NaN,333.559722,NaN,NaN
4,Doctor & Facility,00120,NaN,186.200899,142.635455,NaN


In [22]:
df_avg_pmt_pvt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11179 entries, 0 to 11178
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   payment_type      11179 non-null  object 
 1   hcpcs_code        11179 non-null  object 
 2   2015              9469 non-null   float64
 3   2016              9658 non-null   float64
 4   2017              9752 non-null   float64
 5   change_2017_2015  8379 non-null   float64
dtypes: float64(4), object(2)
memory usage: 524.1+ KB


In [23]:
df_avg_pmt_pvt['change_2017_2016'] = df_avg_pmt_pvt[2017] - df_avg_pmt_pvt[2016]

In [24]:
df_avg_pmt_pvt['change_2016_2015'] = df_avg_pmt_pvt[2016] - df_avg_pmt_pvt[2015]

In [25]:
df_avg_pmt_pvt.head()

year,payment_type,hcpcs_code,2015,2016,2017,change_2017_2015,change_2017_2016,change_2016_2015
0,Doctor & Facility,0008M,NaN,3419.419776,3443.360000,NaN,23.940224,NaN
1,Doctor & Facility,00103,255.076212,128.268843,118.002858,-137.073353,-10.265984,-126.807369
2,Doctor & Facility,00104,111.614526,75.184242,74.647106,-36.967421,-0.537137,-36.430284
3,Doctor & Facility,0010M,NaN,333.559722,NaN,NaN,NaN,NaN
4,Doctor & Facility,00120,NaN,186.200899,142.635455,NaN,-43.565445,NaN


In [26]:
df_avg_pmt_pvt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11179 entries, 0 to 11178
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   payment_type      11179 non-null  object 
 1   hcpcs_code        11179 non-null  object 
 2   2015              9469 non-null   float64
 3   2016              9658 non-null   float64
 4   2017              9752 non-null   float64
 5   change_2017_2015  8379 non-null   float64
 6   change_2017_2016  8843 non-null   float64
 7   change_2016_2015  8682 non-null   float64
dtypes: float64(6), object(2)
memory usage: 698.8+ KB


In [27]:
df_avg_pmt_pvt.tail()

year,payment_type,hcpcs_code,2015,2016,2017,change_2017_2015,change_2017_2016,change_2016_2015
11174,Facility Only,Q4172,NaN,NaN,117.524101,NaN,NaN,NaN
11175,Facility Only,Q9968,NaN,6.697629,3.404030,NaN,-3.293600,NaN
11176,Facility Only,R0070,188.247336,187.518203,180.445074,-7.802262,-7.073129,-0.729132
11177,Facility Only,R0075,81.782458,80.305087,80.649999,-1.132458,0.344913,-1.477371
11178,Facility Only,V2785,3128.787733,3784.231305,3844.634734,715.847001,60.403429,655.443572


In [33]:
df_avg_pmt_pvt['abs_change_2015_2017'] = abs(df_avg_pmt_pvt.change_2017_2015)

In [34]:
df_avg_pmt_pvt.head()

year,payment_type,hcpcs_code,2015,2016,2017,change_2017_2015,change_2017_2016,change_2016_2015,abs_change_2015_2017
0,Doctor & Facility,0008M,NaN,3419.419776,3443.360000,NaN,23.940224,NaN,NaN
1,Doctor & Facility,00103,255.076212,128.268843,118.002858,-137.073353,-10.265984,-126.807369,137.073353
2,Doctor & Facility,00104,111.614526,75.184242,74.647106,-36.967421,-0.537137,-36.430284,36.967421
3,Doctor & Facility,0010M,NaN,333.559722,NaN,NaN,NaN,NaN,NaN
4,Doctor & Facility,00120,NaN,186.200899,142.635455,NaN,-43.565445,NaN,NaN


In [35]:
# melt dataframe
df_avg_pmt_melt = pd.melt(df_avg_pmt_pvt,
                         id_vars = ['payment_type', 'hcpcs_code'],
                         value_vars = ['change_2016_2015',
                                       'change_2017_2016',
                                       'change_2017_2015',
                                       'abs_change_2015_2017'],
                         var_name = 'change')


In [72]:
df_avg_pmt_melt.shape

(44716, 4)

In [68]:

df_avg_pmt_melt.hcpcs_code.nunique()


6763

In [64]:
df_avg_pmt_melt_tot_change = df_avg_pmt_melt.loc[df_avg_pmt_melt['change'] == 'change_2017_2015']

df_avg_pmt_melt_tot_change.head()

,payment_type,hcpcs_code,change,value
0,Doctor & Facility,0008M,change_2017_2015,NaN
1,Doctor & Facility,00103,change_2017_2015,-126.807369
2,Doctor & Facility,00104,change_2017_2015,-36.430284
3,Doctor & Facility,0010M,change_2017_2015,NaN
4,Doctor & Facility,00120,change_2017_2015,NaN


In [73]:
df_avg_pmt_melt_tot_change.shape

(44716, 4)

In [76]:
df_avg_pmt_melt_tot_change.payment_type.value_counts()

Doctor & Facility    19244
Doctor Only          18844
Facility Only         6628
Name: payment_type, dtype: int64

In [69]:
df_avg_pmt_melt_tot_change.hcpcs_code.nunique()

6763

In [59]:
df_avg_pmt_melt_tot_change_both = df_avg_pmt_melt_tot_change.loc[df_avg_pmt_melt_tot_change['payment_type'] == 'Doctor & Facility']

df_avg_pmt_melt_tot_change_both.head()

,payment_type,hcpcs_code,change,value
38107,Doctor & Facility,J7190,change_2017_2015,16273.275534
4569,Doctor & Facility,J7187,change_2017_2015,10338.008318
4683,Doctor & Facility,L8679,change_2017_2015,7633.141951
38220,Doctor & Facility,L8679,change_2017_2015,7519.650179
27041,Doctor & Facility,L8679,change_2017_2015,7519.650179


In [60]:
df_avg_pmt_melt_tot_change_both = df_avg_pmt_melt_tot_change_both[df_avg_pmt_melt_tot_change_both['value'].notna()]
df_avg_pmt_melt_tot_change_both.head()

,payment_type,hcpcs_code,change,value
38107,Doctor & Facility,J7190,change_2017_2015,16273.275534
4569,Doctor & Facility,J7187,change_2017_2015,10338.008318
4683,Doctor & Facility,L8679,change_2017_2015,7633.141951
38220,Doctor & Facility,L8679,change_2017_2015,7519.650179
27041,Doctor & Facility,L8679,change_2017_2015,7519.650179


In [62]:
df_avg_pmt_melt_tot_change_both = df_avg_pmt_melt_tot_change_both.sort_values(by = 'value', ascending = False)
df_avg_pmt_melt_tot_change_both.head(10)

,payment_type,hcpcs_code,change,value
38107,Doctor & Facility,J7190,change_2017_2015,16273.275534
4569,Doctor & Facility,J7187,change_2017_2015,10338.008318
4683,Doctor & Facility,L8679,change_2017_2015,7633.141951
27041,Doctor & Facility,L8679,change_2017_2015,7519.650179
38220,Doctor & Facility,L8679,change_2017_2015,7519.650179
38108,Doctor & Facility,J7192,change_2017_2015,5772.223670
38106,Doctor & Facility,J7187,change_2017_2015,3874.924947
26927,Doctor & Facility,J7187,change_2017_2015,3874.924947
27090,Doctor & Facility,Q2043,change_2017_2015,3340.527942
38269,Doctor & Facility,Q2043,change_2017_2015,3340.527942


In [71]:
df_avg_pmt_melt_tot_change_both.shape


(15287, 4)

In [ ]:
# dead end - need to go back to the groupby code and make it better, duplicate hcpcs_code rows for payment_types